In [1]:
# retrieve text from PDF
from tqdm.notebook import tqdm
from PyPDF2 import PdfReader
from tqdm.notebook import tqdm
from textblob import TextBlob
import numpy as np
import credentials
import spacy  
import re
from collections import Counter
from nltk.corpus import wordnet
import nltk
nltk.download('omw-1.4')
import pandas as pd
import numpy as np
from transformers import pipeline
import pdfplumber
import os
from difflib import SequenceMatcher

/Users/yingyingren/opt/anaconda3/envs/numpy_environment/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-08-02 19:09:20.248931: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/yingyingren/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### Scrapping last 30 quarterly reports

In [2]:
# Define similarity function
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()
# Define headings in the document using word size function
def word_ratio_func(word):
    try:
        word_length = len(word["text"])
        word_bottom = float(word['bottom'])
        word_top = float(word['top'])
        return (word_bottom - word_top), word_length, word["text"]
        
    except:
        return 0, 0, 0

def preprocess_text(texts):

    # preprocess the text
    text = "".join(texts.values()).strip("●").strip("*")
    text = text.split("\n")
    text = [x for x in text if x != '' and x.startswith("Source") == False]
    text = [x[0].replace("●", "") + x[1:] if x[0] == "●" else x for x in text]
    text = [x[0].replace("*", "") + x[1:] if x[0] == "*" else x for x in text]
    text = [x[0].replace("○", "") + x[1:] if x[0] == "○" else x for x in text]
    text = [x[0].replace("1", "") + x[1:] if x[1:3] in ["Q1", "Q2", "Q3", "Q4"] else x for x in text]
    text = text[2:]

    return text


In [3]:
# get file paths for all pdfs
pdf_paths = []
for root, dirs, files in os.walk("ShareholderLetters/"):
    for file in files:
        if file.endswith(".pdf"):
             pdf_paths.append(os.path.join(root, file))

In [4]:
# creating a pdf reader object
pdf_paths = ["ShareholderLetters/FINAL-Q1-16-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-16-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-16-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-16-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-17-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-17-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-17-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-17-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-18-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-18-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-18-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-18-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-19-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-19-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-19-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-19-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-20-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-20-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-20-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-20-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-21-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-21-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-21-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-21-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-22-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q2-22-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q3-22-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q4-22-Shareholder-Letter.pdf",
             "ShareholderLetters/FINAL-Q1-23-Shareholder-Letter.pdf",
            "ShareholderLetters/FINAL-Q2-23-Shareholder-Letter.pdf"]

# Initialize the summarizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

pdf_texts = {}
pdf_headings = {}

if not os.path.exists('Txt'):
    os.makedirs('Txt')

for file_path in tqdm(pdf_paths):

    try:
        
        reader = pdfplumber.open(file_path)

        texts = {}
        texts_v2 = {}
        headings = []
        headings_count = 0

        for page_number in range(0, len(reader.pages)):

            # get the specific page from the pdf file
            page = reader.pages[page_number]
            # extract text from page
            text = page.extract_text()
            # add text to dictionary
            texts[page_number] = text

            # extract headings from page
            words = page.extract_words()
            word_count = 0
            while word_count < len(words):
                # find if the words are large enough to be headings
                word_size, word_length, word_text = word_ratio_func(words[word_count])
                heading = []

                if word_size > 15 and word_length > 1:
                    while True:
                        heading.append(word_text)
                        word_count += 1
                        if word_count >= len(words):
                            break
                        word_size, word_length, word_text = word_ratio_func(words[word_count])
                        if not word_size > 15 and word_length > 1:
                            headings.append(" ".join(heading))
                            word_count += 10
                            break
                headings_count += 1
                word_count += 1
            
            # break if the page covers the reference section
            if re.search("\nReference\n", text):
                break
        
        # preprocess the text
        text = preprocess_text(texts)
        final_text = " ".join(text)

        # export the text to a txt file
        with open("Txt/" + file_path.split("/")[-1].split(".")[0] + ".txt", "w", encoding='utf-8') as f:
            f.write(final_text)

        # add the text to the dictionary
        pdf_texts[file_path.split("/")[-1].split(".")[0]] = final_text
        # add the headings to the dictionary
        pdf_headings[file_path.split("/")[-1].split(".")[0]] = headings

        
    except Exception as e:
        print(e)
        continue

2023-08-02 19:09:28.838741: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFBartForConditionalGeneration.

All the layers of TFBartForConditionalGeneration were initialized from the model checkpoint at facebook/bart-large-cnn.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.


  0%|          | 0/30 [00:00<?, ?it/s]

In [5]:
# view your pdf texts with pdf names as keys
list(pdf_texts.items())

[('FINAL-Q1-16-Shareholder-Letter',
  'We are off to a great start in 2016, expanding the Netflix service to 130 new countries in January and finishing Q1 with over 81 million members on the strength of our fast growing slate of original content. Since 2010, we have offered unlimited streaming for $7.99 per month in the US, and we hope to maintain this affordable entry-price for a long time. As we’ve made vast improvements in our service and content, consumers see even more value and our membership grows. Q1 was no exception and our summary results are below. Q1 Results Net additions in Q1 ‘16 amounted to a record 6.74 million, surpassing our prior high water mark of 5.59 million in Q4 ‘15 and against 4.88 million in the year-ago period. Of our 81.5 million members, 42% are now outside of the US. Operating income was $49 million vs. $97 million in the year ago quarter, as we continue to invest in international markets. Performance in the quarter benefited from the launch of several ser

### Tokenized the earning reports

In [6]:
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

# Tokenize all the reports
tokenized_reports = {}

# Loop through each report in pdf_texts
for report_name, report_text in pdf_texts.items():
    # Use the nlp.pipe method to tokenize the report_text
    tokenized_report = [doc for doc in nlp.pipe([report_text])]
    # Add tokenized_report to tokenized_reports
    tokenized_reports[report_name] = tokenized_report

# Print the tokenized reports
for report_name, tokenized_report in tokenized_reports.items():
    print(f"Report name: {report_name}")
    for doc in tokenized_report:
        print([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])  
    print("\n")

Report name: FINAL-Q1-16-Shareholder-Letter
['great', 'start', '2016', 'expand', 'Netflix', 'service', '130', 'new', 'country', 'January', 'finish', 'Q1', '81', 'million', 'member', 'strength', 'fast', 'grow', 'slate', 'original', 'content', '2010', 'offer', 'unlimited', 'streaming', '$', '7.99', 'month', 'hope', 'maintain', 'affordable', 'entry', 'price', 'long', 'time', 'vast', 'improvement', 'service', 'content', 'consumer', 'value', 'membership', 'grow', 'Q1', 'exception', 'summary', 'result', 'Q1', 'result', 'net', 'addition', 'Q1', '16', 'amount', 'record', '6.74', 'million', 'surpass', 'prior', 'high', 'water', 'mark', '5.59', 'million', 'Q4', '15', '4.88', 'million', 'year', 'ago', 'period', '81.5', 'million', 'member', '42', 'outside', 'operate', 'income', '$', '49', 'million', 'vs.', '$', '97', 'million', 'year', 'ago', 'quarter', 'continue', 'invest', 'international', 'market', 'performance', 'quarter', 'benefit', 'launch', 'series', 'include', 'make', 'Murderer', '\u200b', 

### Find the most frequent mentioned financial keywords in each report

In [7]:
def find_related_financial_keywords(tokenized_reports):
    # financial_terms.txt is a list of financial keywords from Tilburg University
    with open('financial_terms.txt', 'r') as f:
        financial_keywords = [line.strip() for line in f]

    # Find all synsets related to financial keywords
    financial_synsets = [wordnet.synsets(keyword) for keyword in financial_keywords]

    # Flatten the list of synsets
    financial_synsets = [synset for sublist in financial_synsets for synset in sublist]

    # Find all lemmas for these synsets
    financial_lemmas = [lemma.name() for synset in financial_synsets for lemma in synset.lemmas()]

    # Initialize a dictionary to store the most frequent financial keywords for each document
    financial_keywords_counts_dict = {}

    # Iterate over each document in tokenized_reports
    for report_name, tokenized_report in tokenized_reports.items():

        # For each tokenized report, get a list of all tokens
        all_tokens = [token.text for doc in tokenized_report for token in doc]

        # Filter tokens to only include those in the financial lemmas list
        financial_tokens = [token for token in all_tokens if token in financial_lemmas]

        # Use Counter to count each financial token's occurrences
        financial_token_counts = Counter(financial_tokens)

        # Only keep the top 10 most frequent financial words
        top_10_financial_words = financial_token_counts.most_common(10)

        # Add the result to the dictionary
        financial_keywords_counts_dict[report_name] = top_10_financial_words

    return financial_keywords_counts_dict

# Call the function
financial_keywords_counts = find_related_financial_keywords(tokenized_reports)

# Print the results
for report_name, financial_keywords_counts in financial_keywords_counts.items():
    print(f"Report name: {report_name}")
    for word, count in financial_keywords_counts:
        print(f"Word: {word}, Count: {count}")
    print("\n")


Report name: FINAL-Q1-16-Shareholder-Letter
Word: be, Count: 10
Word: net, Count: 8
Word: second, Count: 7
Word: forecast, Count: 6
Word: world, Count: 6
Word: value, Count: 5
Word: make, Count: 5
Word: market, Count: 5
Word: global, Count: 5
Word: release, Count: 4


Report name: FINAL-Q2-16-Shareholder-Letter
Word: forecast, Count: 12
Word: net, Count: 10
Word: be, Count: 8
Word: global, Count: 7
Word: profit, Count: 6
Word: world, Count: 6
Word: market, Count: 5
Word: Canada, Count: 4
Word: revenue, Count: 3
Word: gross, Count: 3


Report name: FINAL-Q3-16-Shareholder-Letter
Word: forecast, Count: 10
Word: revenue, Count: 9
Word: global, Count: 7
Word: net, Count: 7
Word: be, Count: 7
Word: world, Count: 3
Word: release, Count: 3
Word: currency, Count: 2
Word: total, Count: 2
Word: profit, Count: 2


Report name: FINAL-Q4-16-Shareholder-Letter
Word: net, Count: 14
Word: global, Count: 11
Word: be, Count: 11
Word: forecast, Count: 8
Word: revenue, Count: 5
Word: world, Count: 5
Word:

In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

def find_top10_frequent_words(tokenized_reports):
    # Select the two reports
    reports = ['FINAL-Q1-23-Shareholder-Letter', 'FINAL-Q2-23-Shareholder-Letter']

    # Initialize a dictionary to store the most frequent words for each selected report
    frequent_words_counts_dict = {}

    # Iterate over each selected report
    for report_name in reports:
        if report_name in tokenized_reports:
            tokenized_report = tokenized_reports[report_name]

            # For each tokenized report, get a list of all tokens
            all_tokens = [token.text.lower() for doc in tokenized_report for token in doc]

            # Filter out punctuation, stop words, and other non-alphabetic tokens
            all_tokens = [token for token in all_tokens if token.isalpha() and not nlp.vocab[token].is_stop]

            # Use Counter to count each token's occurrences
            token_counts = Counter(all_tokens)

            # Only keep the top 10 most frequent words
            top_10_words = token_counts.most_common(10)

            # Add the result to the dictionary
            frequent_words_counts_dict[report_name] = top_10_words

    return frequent_words_counts_dict

# Call the function
frequent_words_counts = find_top10_frequent_words(tokenized_reports)

# Print the results
for report_name, frequent_words_counts in frequent_words_counts.items():
    print(f"Report name: {report_name}")
    for word, count in frequent_words_counts:
        print(f"Word: {word}, Count: {count}")
    print("\n")


Report name: FINAL-Q1-23-Shareholder-Letter
Word: forexample, Count: 4
Word: x, Count: 3
Word: memberadvertisingeconomics, Count: 2
Word: forecast, Count: 2
Word: revenue, Count: 2
Word: y, Count: 2
Word: business, Count: 2
Word: similarly, Count: 2
Word: monetization, Count: 2
Word: summary, Count: 1


Report name: FINAL-Q2-23-Shareholder-Letter
Word: forecast, Count: 3
Word: inmay, Count: 2
Word: revenue, Count: 2
Word: y, Count: 2
Word: f, Count: 2
Word: cash, Count: 2
Word: ourrevenuebase, Count: 1
Word: revenueineachregionisnowhigherthanpre, Count: 1
Word: launch, Count: 1
Word: withsign, Count: 1




### Split documents into sentences

In [9]:
# Initialize a dictionary to store the joined sentences for each report.
joined_sentences = {}

# Iterate over each report in pdf_texts.
for report_name, report_text in pdf_texts.items():

    # Split the report text into sentences.
    sentences = nlp(report_text).sents

    # Initialize a list to hold the tokenized sentences for this report.
    tokenized_report_sentences = []

    # Iterate over each sentence.
    for sentence in sentences:
        # Tokenize, lemmatize, and remove stop words and punctuation.
        tokenized = [token.lemma_ for token in sentence if not token.is_stop and not token.is_punct]
        # Add the tokenized sentence to the list.
        tokenized_report_sentences.append(tokenized)

    # Join each tokenized sentence into a single string, and store them in a list.
    joined_report_sentences = [' '.join(sentence) for sentence in tokenized_report_sentences]

    # Add the joined sentences for this report to joined_sentences.
    joined_sentences[report_name] = joined_report_sentences

# Print the joined sentences for each report.
for report_name, joined_report_sentences in joined_sentences.items():
    print(f"Report name: {report_name}")
    print(joined_report_sentences)
    print("\n")

Report name: FINAL-Q1-16-Shareholder-Letter
['great start 2016 expand Netflix service 130 new country January finish Q1 81 million member strength fast grow slate original content', '2010 offer unlimited streaming $ 7.99 month hope maintain affordable entry price long time', 'vast improvement service content consumer value membership grow', 'Q1 exception summary result', 'Q1 result net addition Q1 16 amount record 6.74 million surpass prior high water mark 5.59 million Q4 15 4.88 million year ago period', '81.5 million member 42 outside', 'operate income $ 49 million vs. $ 97 million year ago quarter continue invest international market', 'performance quarter benefit launch series include make Murderer \u200b \u200b late December debut Fuller House February House Cards Season 4 March Daredevil Season 2 \u200b \u200b \u200b \u200b \u200b \u200b March complement diverse mix new film documentary series kid show available globally', 'add 2.23 million member Q1 2.28 million year ago 2.25 ye

### Extract sentences with keywords

In [43]:
def analyze_sentences_with_keywords(joined_sentences, keywords):
    # Initialize a dictionary to store the sentences for each report
    sentences_with_keywords_dict = {}

    # Iterate over each report
    for report_name, joined_sentences in joined_sentences.items():
        # For each report, find sentences containing each keyword
        sentences_with_keywords = {keyword: [] for keyword in keywords}

        for sentence in joined_sentences:
            for keyword in keywords:
                if keyword in sentence.lower():
                    # Add the sentence to the corresponding keyword list
                    sentences_with_keywords[keyword].append(sentence)

        # Add the result to the dictionary
        sentences_with_keywords_dict[report_name] = sentences_with_keywords

    return sentences_with_keywords_dict

# Call the function
keywords = ['revenue', 'forecast', 'profit']
sentences_with_keywords = analyze_sentences_with_keywords(joined_sentences, keywords)

# Print the results
for report_name, keywords_sentences_dict in sentences_with_keywords.items():
    print(f"Report name: {report_name}")
    for keyword, sentences in keywords_sentences_dict.items():
        print(f"Keyword: {keyword}")
        for sentence in sentences:
            print(f"Sentence: {sentence}")
        print("\n")


Report name: FINAL-Q1-16-Shareholder-Letter
Keyword: revenue
Sentence: revenue rise 18 year year reflect 14 growth average pay membership 3 increase ARPU
Sentence: international revenue increase 57 year year include negative impact currency -$82 million y y basis


Keyword: forecast
Sentence: forecast quarter low 1.75 million underestimate positive acquisition impact major original content debut
Sentence: Q1 international contribution loss $ 104 million come well forecast timing content spend
Sentence: q2 Forecast
Sentence: reminder guidance provide actual internal forecast time report
Sentence: Q2 net add forecast 0.5 million line prior year 0.5 0.6 0.6 0.9 million 2012 2015 take account modest impact beginning un grandfathering
Sentence: 2our international forecast few net add prior year tough comparison Australia New Zealand launch
Sentence: Ex ANZ international net add forecast quarter
Sentence: forecast flat Y Y contribution margin Q2 large slate content release associated marketi

#### sentiment analysis for each sentence

In [46]:
def analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords):
    # Initialize the sentiment analysis pipeline
    sentiment_analysis = pipeline("sentiment-analysis")

    # Initialize a dictionary to store the sentences and their sentiment scores for each report
    sentiment_results_dict = {}

    # Iterate over each report
    for report_name, sentences in joined_sentences.items():

        # Initialize a dictionary to store the sentiment analysis results for the current report
        report_sentiment_results = {keyword: [] for keyword in keywords}

        for sentence in sentences:
            if len(sentence) < 512: 
                for keyword in keywords:
                    if keyword in sentence.lower():
                        # Analyze the sentiment of the sentence
                        sentiment_result = sentiment_analysis(sentence)[0]
                        report_sentiment_results[keyword].append(sentiment_result)

        # Add the results to the dictionary
        sentiment_results_dict[report_name] = report_sentiment_results

    return sentiment_results_dict

keywords = ['revenue', 'forecast', 'profit']
sentences_with_keywords_and_sentiment = analyze_sentiment_of_sentences_with_keywords(joined_sentences, keywords)
sentences_with_keywords_and_sentiment

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


{'FINAL-Q1-16-Shareholder-Letter': {'revenue': [{'label': 'POSITIVE',
    'score': 0.9847038984298706},
   {'label': 'NEGATIVE', 'score': 0.9788206219673157}],
  'forecast': [{'label': 'NEGATIVE', 'score': 0.9905502796173096},
   {'label': 'NEGATIVE', 'score': 0.9981833100318909},
   {'label': 'POSITIVE', 'score': 0.9054474830627441},
   {'label': 'POSITIVE', 'score': 0.9201352596282959},
   {'label': 'POSITIVE', 'score': 0.7879024147987366},
   {'label': 'NEGATIVE', 'score': 0.9536502361297607},
   {'label': 'NEGATIVE', 'score': 0.9708260893821716},
   {'label': 'NEGATIVE', 'score': 0.998506486415863}],
  'profit': [{'label': 'NEGATIVE', 'score': 0.8029835224151611},
   {'label': 'NEGATIVE', 'score': 0.9757313132286072},
   {'label': 'NEGATIVE', 'score': 0.990978479385376}]},
 'FINAL-Q2-16-Shareholder-Letter': {'revenue': [{'label': 'NEGATIVE',
    'score': 0.859438419342041},
   {'label': 'POSITIVE', 'score': 0.9980989098548889},
   {'label': 'POSITIVE', 'score': 0.9994509816169739}]

#### total sentiment scores for each report

In [40]:
# Initialize a dictionary to hold total scores for each keyword in each report
total_scores = {report: {keyword: 0 for keyword in keywords} for report in sentences_with_keywords_and_sentiment.keys()}

# Calculate total scores for each keyword in each report
for report_name, keywords_dict in sentences_with_keywords_and_sentiment.items():
    for keyword, sentiments in keywords_dict.items():
        for sentiment in sentiments:
            # If the sentiment is POSITIVE, add the score
            # If the sentiment is NEGATIVE, subtract the score
            if sentiment['label'] == 'POSITIVE':
                total_scores[report_name][keyword] += sentiment['score']
            else:
                total_scores[report_name][keyword] -= sentiment['score']

# Convert the total_scores to a DataFrame
df = pd.DataFrame(total_scores).T
df.reset_index(inplace=True)
df.columns = ['report_name', 'revenue_score', 'forecast_score', 'profit_score']

In [41]:
df

,report_name,revenue_score,forecast_score,profit_score
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000


In [47]:
# Add a new column 'total_score'
df['total_score'] = df['revenue_score'] + df['forecast_score'] + df['profit_score']

# Print the results
df

,report_name,revenue_score,forecast_score,profit_score,total_score
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693,-5.062041
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838,-2.888572
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956,3.744891
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570,4.720181
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773,7.942729
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801,3.379399
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840,4.018302
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556,-4.544982
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000,1.933051
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000,-6.784108


### Stock price processing

In [48]:
stock_price = pd.read_csv("NFLX.csv")
stock_price.loc[:, "open_close_diff"] = stock_price.Close - stock_price.Open
stock_price

,Quater,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff
0,FINAL-Q2-23-Shareholder-Letter,19/07/2023,476.860000,485.000000,470.000000,477.590000,477.590000,20210900,0.730000
1,FINAL-Q1-23-Shareholder-Letter,18/04/2023,335.000000,337.190002,330.500000,333.700012,333.700012,17944500,-1.299988
2,FINAL-Q4-22-Shareholder-Letter,19/01/2023,322.570007,324.890015,313.390015,315.779999,315.779999,18008200,-6.790008
3,FINAL-Q3-22-Shareholder-Letter,18/10/2022,249.800003,250.369995,237.729996,240.860001,240.860001,25776700,-8.940002
4,FINAL-Q2-22-Shareholder-Letter,19/07/2022,193.020004,201.970001,188.399994,201.630005,201.630005,28178700,8.610001
5,FINAL-Q1-22-Shareholder-Letter,19/04/2022,333.220001,351.679993,333.220001,348.609985,348.609985,20906900,15.389984
6,FINAL-Q4-21-Shareholder-Letter,20/01/2022,517.750000,526.640015,506.929993,508.250000,508.250000,12659000,-9.500000
7,FINAL-Q3-21-Shareholder-Letter,19/10/2021,636.969971,641.000000,632.299988,639.000000,639.000000,7633100,2.030029
8,FINAL-Q2-21-Shareholder-Letter,20/07/2021,526.070000,536.640000,520.300000,531.050000,531.050000,"6,930,400",4.980000
9,FINAL-Q1-21-Shareholder-Letter,20/04/2021,554.420000,563.560000,546.300000,549.570000,549.570000,"11,257,600",-4.850000


### match the polarity score with open_close_diff 

In [50]:
# Merge sentiment_scores_pivot and stock_price DataFrames
final_df = pd.merge(df, stock_price, left_on='report_name', right_on='Quater')
final_df = final_df.drop(columns=['Quater'])
# Print the results
final_df

,report_name,revenue_score,forecast_score,profit_score,total_score,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693,-5.062041,18/04/2016,109.900002,110.699997,106.019997,108.400002,108.400002,32816400,-1.500000
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838,-2.888572,18/07/2016,98.430000,99.839996,97.239998,98.809998,98.809998,28669700,0.379998
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956,3.744891,17/10/2016,100.500000,100.730003,98.379997,99.800003,99.800003,26589500,-0.699997
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570,4.720181,18/01/2017,133.210007,133.649994,131.059998,133.259995,133.259995,16168600,0.049988
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773,7.942729,17/04/2017,144.429993,147.320007,144.429993,147.250000,147.250000,16364700,2.820007
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801,3.379399,17/07/2017,162.910004,163.550003,160.020004,161.699997,161.699997,17496700,-1.210007
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840,4.018302,16/10/2017,201.949997,202.830002,197.860001,202.679993,202.679993,22105400,0.729996
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556,-4.544982,22/01/2018,222.000000,227.789993,221.199997,227.580002,227.580002,17703300,5.580002
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000,1.933051,16/04/2018,315.989990,316.100006,304.000000,307.779999,307.779999,20307900,-8.209991
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000,-6.784108,16/07/2018,398.980011,403.359985,391.750000,400.480011,400.480011,22960000,1.500000


In [51]:
final_df['match'] = np.sign(final_df['total_score']) == np.sign(final_df['open_close_diff'])

# Print the results
final_df

,report_name,revenue_score,forecast_score,profit_score,total_score,Date,Open,High,Low,Close,Adj Close,Volume,open_close_diff,match
0,FINAL-Q1-16-Shareholder-Letter,0.005883,-2.298231,-2.769693,-5.062041,18/04/2016,109.900002,110.699997,106.019997,108.400002,108.400002,32816400,-1.500000,True
1,FINAL-Q2-16-Shareholder-Letter,1.138111,-2.341845,-1.684838,-2.888572,18/07/2016,98.430000,99.839996,97.239998,98.809998,98.809998,28669700,0.379998,False
2,FINAL-Q3-16-Shareholder-Letter,0.495398,0.301537,2.947956,3.744891,17/10/2016,100.500000,100.730003,98.379997,99.800003,99.800003,26589500,-0.699997,False
3,FINAL-Q4-16-Shareholder-Letter,2.965716,-0.221106,1.975570,4.720181,18/01/2017,133.210007,133.649994,131.059998,133.259995,133.259995,16168600,0.049988,True
4,FINAL-Q1-17-Shareholder-Letter,2.983195,2.973760,1.985773,7.942729,17/04/2017,144.429993,147.320007,144.429993,147.250000,147.250000,16364700,2.820007,True
5,FINAL-Q2-17-Shareholder-Letter,1.336397,-1.088799,3.131801,3.379399,17/07/2017,162.910004,163.550003,160.020004,161.699997,161.699997,17496700,-1.210007,False
6,FINAL-Q3-17-Shareholder-Letter,1.987605,1.504858,0.525840,4.018302,16/10/2017,201.949997,202.830002,197.860001,202.679993,202.679993,22105400,0.729996,True
7,FINAL-Q4-17-Shareholder-Letter,-2.896049,-1.853489,0.204556,-4.544982,22/01/2018,222.000000,227.789993,221.199997,227.580002,227.580002,17703300,5.580002,False
8,FINAL-Q1-18-Shareholder-Letter,1.513936,0.419115,0.000000,1.933051,16/04/2018,315.989990,316.100006,304.000000,307.779999,307.779999,20307900,-8.209991,False
9,FINAL-Q2-18-Shareholder-Letter,-1.327654,-5.456454,0.000000,-6.784108,16/07/2018,398.980011,403.359985,391.750000,400.480011,400.480011,22960000,1.500000,False


In [57]:
counts = final_df['match'].value_counts()
counts

True     16
False    14
Name: match, dtype: int64

In [60]:
count_true = final_df['match'].sum()
accuracy= count_true/30
accuracy

0.5333333333333333